In [ ]:
from self_harm_triage_notes.config import data_interim_dir, data_proc_dir, data_pred_dir, models_dir
from self_harm_triage_notes.text_utils import *
from self_harm_triage_notes.viz_utils import *
import numpy as np
import pandas as pd

# Pandas settings
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100

In [2]:
data_filename = "rmh_2012_2017_dev"
# Load selected features
with open(models_dir / (data_filename + "_selected_fts.txt"), 'r') as f:
    selected_features_rmh = f.read().split()
len(selected_features_rmh)

648

In [3]:
data_filename = "lvrh_2012_2022"
# Load selected features
with open(models_dir / (data_filename + "_selected_fts.txt"), 'r') as f:
    selected_features_lvrh = f.read().split()
len(selected_features_lvrh)

574

In [6]:
selected_features_rmh

['abdo',
 'abilify',
 'abscond',
 'absconded',
 'abuse',
 'abusive',
 'accom',
 'acid',
 'acs',
 'active',
 'addiction',
 'adhd',
 'adipose',
 'admits',
 'advil',
 'af',
 'afebrile',
 'affect',
 'aggressive',
 'agitated',
 'ago',
 'alcohol',
 'alprazolam',
 'altercation',
 'ami',
 'amitriptyline',
 'amounts',
 'ams',
 'analgesia',
 'ane',
 'anger',
 'ankle',
 'antidepressant',
 'antidepressants',
 'anxiety',
 'anymore',
 'approx',
 'argument',
 'argumentative',
 'arms',
 'arterial',
 'assoc',
 'associated',
 'asylum',
 'attempt',
 'attempted',
 'attempts',
 'attendance',
 'auditory',
 'av',
 'avanza',
 'baclofen',
 'balcony',
 'bandaged',
 'beech',
 'beer',
 'beers',
 'behavioural',
 'ben',
 'bends',
 'benzodiazepine',
 'benztropine',
 'bib',
 'bibp',
 'bipolar',
 'blade',
 'blades',
 'bleach',
 'borderline',
 'bottle',
 'bottles',
 'bourbon',
 'bowel',
 'box',
 'boxes',
 'boyfriend',
 'bpad',
 'bpd',
 'brand',
 'break',
 'breakdown',
 'breakup',
 'bridge',
 'brought',
 'bulimia',
 'bu

In [7]:
selected_features_lvrh

['abdo',
 'abscond',
 'absconded',
 'abuse',
 'abusive',
 'accom',
 'accommodation',
 'achy',
 'active',
 'addict',
 'adipose',
 'admits',
 'afebrile',
 'affect',
 'aggressive',
 'agitated',
 'alcohol',
 'alleged',
 'allegedly',
 'alleges',
 'alprazolam',
 'altercation',
 'ambulance',
 'analgesia',
 'anger',
 'angry',
 'ankle',
 'answer',
 'answering',
 'antidepressants',
 'anxiety',
 'anymore',
 'appropriate',
 'appropriately',
 'approx',
 'area',
 'argument',
 'arms',
 'arrival',
 'assoc',
 'associated',
 'attempt',
 'attempted',
 'attempts',
 'attendance',
 'av',
 'avanza',
 'avoiding',
 'avoids',
 'awake',
 'baclofen',
 'bandaged',
 'beer',
 'beers',
 'belong',
 'berry',
 'bib',
 'biba',
 'bipolar',
 'blade',
 'bleach',
 'blunted',
 'borderline',
 'bottle',
 'bottles',
 'bourbon',
 'bowel',
 'bowels',
 'boyfriend',
 'bpd',
 'breakdown',
 'broke',
 'brought',
 'bullied',
 'bullying',
 'c/o',
 'called',
 'calm',
 'calmer',
 'campral',
 'cannabis',
 'cans',
 'capsicum',
 'carer',
 'ca

In [8]:
set(selected_features_rmh) - set(selected_features_lvrh)

{'abilify',
 'acid',
 'acs',
 'addiction',
 'adhd',
 'advil',
 'af',
 'ago',
 'ami',
 'amitriptyline',
 'amounts',
 'ams',
 'ane',
 'antidepressant',
 'argumentative',
 'arterial',
 'asylum',
 'auditory',
 'balcony',
 'beech',
 'behavioural',
 'ben',
 'bends',
 'benzodiazepine',
 'benztropine',
 'bibp',
 'blades',
 'box',
 'boxes',
 'bpad',
 'brand',
 'break',
 'breakup',
 'bridge',
 'bulimia',
 'burning',
 'butchers',
 'butter',
 'ca',
 'cade',
 'caffeine',
 'carbon',
 'catt',
 'ccf',
 'champagne',
 'chol',
 'clonidine',
 'cms',
 'cocaine',
 'codeine',
 'codral',
 'communication',
 'copd',
 'cp',
 'crying',
 'ct',
 'cub',
 'cubital',
 'cutter',
 'death',
 'dep',
 'depressive',
 'difficulty',
 'direct',
 'distressed',
 'dm',
 'dosage',
 'doses',
 'drowning',
 'ds',
 'dsh',
 'ecatt',
 'enalapril',
 'endone',
 'epigastric',
 'ett',
 'evasive',
 'extra',
 'face',
 'facial',
 'febrile',
 'fever',
 'fevers',
 'fluctuating',
 'forte',
 'forthcoming',
 'g',
 'gambling',
 'gamma',
 'generalise

___
# Patient cohort

In [ ]:
# Load data
data_filename = "rmh_2012_2022" # rmh_2012_2022, lvrh_2012_2022
site = "RMH"
df = pd.read_parquet(data_interim_dir / (data_filename + "_cleaned.parquet"), engine="pyarrow")

# Remove data from 2022
df = df[df.year < 2022].copy()

df.shape

In [ ]:
df

In [ ]:
# Number of presentations 
df.year.value_counts().agg(['mean', 'std']).round()

In [ ]:
# Age
df.age.describe().round()

In [ ]:
# Sex
df.sex.value_counts(normalize=True) * 100

In [ ]:
# Arrival method
df.arrival_method.value_counts(normalize=True) * 100

In [ ]:
def plot_proportions(df, col, title, legend=True):
    tmp = df.groupby('SH')[col].value_counts(normalize=True).reset_index()
    tmp.proportion = tmp.proportion*100

    plt.rcParams['figure.figsize'] = (3, 3)

    n_cat = len(df[col].cat.categories)

    palette = sns.color_palette('tab10', n_cat)

    for i in range(n_cat):
        sns.barplot(x='SH', y='proportion', data=tmp[tmp[col].isin(df[col].cat.categories[i:])], 
                    estimator=sum, errorbar=None, 
                    label=df[col].cat.categories[i].capitalize(), color=palette[i]);

    plt.xticks(ticks=(0,1), labels=('Negative', 'Positive'));
    plt.yticks(ticks=(0, 50, 100));
    plt.xlabel('Self-harm');
    plt.ylabel('Percentage');
    if legend:
        plt.legend(bbox_to_anchor=(1, 1.035));
    else:
        plt.legend(None)
    plt.title(title);

In [ ]:
plot_proportions(df, 'sex', site)

In [ ]:
plot_proportions(df, 'arrival_method', site)

___
# Length of triage notes

In [ ]:
# Load data
data_filename = "rmh_2012_2022" # rmh_2012_2022, lvrh_2012_2022
site = "RMH"
df = pd.read_parquet(data_interim_dir / (data_filename + "_cleaned.parquet"), engine="pyarrow")

# Remove data from 2022
df = df[df.year < 2022].copy()

df.shape

In [ ]:
df.length.describe()

In [ ]:
sns.lineplot(x='year', y='length', 
            #  hue=df.SH.map({0: 'Negative', 1: 'Positive'}), 
             data=df, 
            #  estimator='mean', errorbar='sd', 
             lw=2, 
            #  palette={'Negative': sns.color_palette('tab10')[7], 
            #           'Positive': sns.color_palette('tab10')[3]}
                      )

In [ ]:
def plot_length_over_time(df, title, annotate_dev=True):
    """Plot character length of triage notes over time"""
    
    plt.rcParams['figure.figsize'] = (df.quarter.nunique() * 12 / 40, 3)

    sns.lineplot(x='quarter', y='length', hue=df.SH.map({0: 'Negative', 1: 'Positive'}), data=df, 
                 estimator='mean', errorbar='sd', 
                 lw=2, palette={'Negative': sns.color_palette('tab10')[7], 
                                'Positive': sns.color_palette('tab10')[3]})
    
    if annotate_dev:
        # Horisontal line: dev and test sets
        plt.plot([0, 23], [300, 300], marker='s', markevery=True, color=sns.color_palette('tab20c')[-4]);
    
    # Axes limits, ticks, and labels
    plt.ylim([50, 600]);
    plt.xticks(rotation=45, 
               ticks=range(0, df.quarter.nunique(), 2), 
               labels=[format_quarter(q) for q in df.quarter.cat.categories.astype(str) 
                       if q.endswith('1') or q.endswith('3')]);
    plt.xlabel("Arrival date");
    plt.ylabel("Character length");
    plt.legend(title='Self-harm');

    # Title
    plt.title(title);

In [ ]:
plot_length_over_time(df, site)

___
# Proportions of SH and SI

In [ ]:
# Load data
data_path = "../datasets/"
data_filename = "lvrh_2012_2022_cleaned" # rmh_2012_2022_cleaned, lvrh_2012_2022_cleaned
site = "LRH"
df = pd.read_csv(data_path + data_filename + ".csv")

# Remove data from 2022
df = df[df.year < 2022].copy()

df.shape

In [ ]:
(df.SH.value_counts(dropna=False, normalize=True) * 100).round(1)

In [ ]:
(df.SI.value_counts(dropna=False, normalize=True) * 100).round(1)

In [ ]:
def plot_presentations_over_time(df, title, annotate_dev=True, simple_palette=True):
    """
    Plot the number of presentations and SH/SI rates per quarter.
    """
    # Convert quarter to categorical
    df.quarter = df.quarter.astype('category')

    # Create subplots
    plt.rcParams['figure.figsize'] = (df.quarter.nunique() * 12 / 40, 6)
    _, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    # Barplot: Numper of ED presentations
    if simple_palette:
        sns.countplot(x='quarter', data=df, 
                      color=sns.color_palette('tab20c')[-1],
                      ax=ax1);
    else:
        palette = {year: sns.color_palette('tab20c')[-2] for year in range(2012, 2018)}
        palette.update({year: sns.color_palette('tab20c')[-1] for year in range(2018, 2022)})
        sns.countplot(x='quarter', data=df, 
                      palette=palette, hue='year', legend=False,
                      ax=ax1);
    
    if annotate_dev:
        # Horisontal line: dev and test sets
        plt.plot([-0.5, 23.5], [2.2, 2.2], marker='s', markevery=True, color=sns.color_palette('tab20c')[-4]);


    # Vertical line: start of covid
    plt.axvline(32.5, 0, 1, color=sns.color_palette('tab20c')[-4], ls='--');
    
    # Axes limits, ticks, and labels
    ax1.set_xticks(rotation=45, 
                   ticks=range(0, df.quarter.nunique(), 2), 
                   labels=[format_quarter(q) for q in df.quarter.cat.categories.astype(str) 
                           if q.endswith('1') or q.endswith('3')]);
    ax1.set_xlabel("Arrival date");
    ax1.set_ylabel("# ED presentations");

    # Lineplot: SH rate per quarter
    sns.lineplot(df.groupby(df.quarter.cat.codes).apply(lambda x: 
                                                    x.SH.sum() / x.shape[0] * 100), 
                                                    color=sns.color_palette("tab10")[3], lw=2, 
                                                    label="Self-harm", ax=ax2);
    # Lineplot: SI rate per quarter
    sns.lineplot(df.groupby(df.quarter.cat.codes).apply(lambda x: 
                                                    x.SI.sum() / x.shape[0] * 100), 
                                                    color=sns.color_palette("tab10")[9], lw=2, 
                                                    label="Suicidal ideation", ax=ax2);
    # Axes limits, ticks, and labels
    ax2.set_ylim([0, 2.8]);
    ax2.set_ylabel("% cases");

    # Title
    plt.title(title);

    # Save plot
    plt.savefig("../results/" + title + " presentations and cases per quarter.jpeg", bbox_inches='tight', dpi=300);

In [ ]:
plot_presentations_over_time(df, site, annotate_dev=False, simple_palette=True)

___
# Number of unique tokens

## Development data

### Load data

In [ ]:
# Load dev data
dev_data_filename = "rmh_2012_2017_dev"
name = "Development"
df_dev = pd.read_parquet(data_proc_dir / (dev_data_filename + "_normalised.parquet"), engine="pyarrow")

# Convert quarter to categorical
df_dev.quarter = df_dev.quarter.astype('category')
df_dev.shape

In [ ]:
# ED vocabulary
vocab_filename = "rmh_2012_2017_dev_amt6"
# Load the ED vocabulary
vocab = load_vocab(vocab_filename)

In [ ]:
# Load selected features
with open(models_dir / (dev_data_filename + "_selected_fts.txt"), 'r') as f:
    selected_features = f.read().split()
len(selected_features)

In [ ]:
dev_counts = count_vocab_tokens_in_data(df_dev.entities, selected_features)

In [ ]:
# with open("../datasets/spelling_correction/negation_terms.txt", 'r') as f:
#     negation_terms = f.read().split()
# len(negation_terms)

### Temporal changes

In [ ]:
plot_dim_over_time(df_dev, name)

In [ ]:
plot_token_overlap_over_time(df_dev, vocab, name)

In [ ]:
plot_dim_reduction_over_time(df_dev, 'preprocessed_triage_note', 'entities', name)

In [ ]:
plot_selected_fts_over_time(df_dev, selected_features, name)

In [ ]:
plot_divergence_over_time(df_dev, dev_counts, selected_features, name)

## Unseen data

### Load data

In [ ]:
# Load unseen data
unseen_data_filename = "rmh_2012_2017_test" # rmh_2012_2017_test, rmh_2018_2022, lvrh_2012_2022
name = "Test"
df = pd.read_parquet(data_proc_dir / (unseen_data_filename + "_normalised.parquet"), engine="pyarrow")

# Remove data from 2022
df = df[df.year < 2022].copy()
# Convert quarter to categorical
df.quarter = df.quarter.astype('category')
df.shape

### Temporal changes

In [ ]:
# 1 & 4. How many unique tokens were there 
# before and after text normalisation?
plot_dim_over_time(df, name)

In [ ]:
# 2 & 5. How similar were the notes to the normalised 
# development data before and after text normalisation?
plot_token_overlap_over_time(df, vocab, name)

In [ ]:
# 3. How effective was text normalisation 
# in dimensionality reduction? 
plot_dim_reduction_over_time(df, 'preprocessed_triage_note', 'entities', name)

In [ ]:
# 6. How many of the selected features were 
# present in the normalised notes?
plot_selected_fts_over_time(df, selected_features, name)

In [ ]:
# 7. Distribution of selected features
plot_divergence_over_time(df, dev_counts, selected_features, name)

___
# Overall predictions

In [ ]:
# Load data
data_filename = "rmh_2012_2017_dev" # rmh_2012_2017_test rmh_2018_2022, lvrh_2012_2022
title = "Development"
df = pd.read_parquet(data_pred_dir / (data_filename + "_predicted.parquet"), engine="pyarrow")

# Remove data from 2022
df = df[df.year < 2022].copy()

df.shape

In [ ]:
plt_predicted_proba(df, title)

___
# Predcitions over time

In [ ]:
# Load data
data_filename = "rmh_2012_2017_dev" # rmh_2012_2017_test rmh_2018_2022, lvrh_2012_2022
title = "Development"
df = pd.read_parquet(data_pred_dir / (data_filename + "_predicted.parquet"), engine="pyarrow")

# Remove data from 2022
df = df[df.year < 2022].copy()

df.shape

In [ ]:
plot_scores_over_time(df, title)

# Char length distributions

In [ ]:
min_len = 124
max_len = 277

In [ ]:
n_notes = ((df1.length >= min_len) & (df1.length <= max_len)). sum()
n_notes, (n_notes / df1.shape[0] * 100).round()

In [ ]:
n_notes = ((df2.length >= min_len) & (df2.length <= max_len)). sum()
n_notes, (n_notes / df2.shape[0] * 100).round()

In [ ]:
sns.histplot(x='length', data=df1[(df1.length >= min_len) & (df1.length <= max_len)].sample(10000), binwidth=10);
sns.histplot(x='length', data=df2[(df2.length >= min_len) & (df2.length <= max_len)].sample(10000), binwidth=10);
plt.xlim([0,1000])

# Language of triage notes

In [ ]:
# ED vocabulary
vocab_filename = "lvrh_2012_2017_dev_amt5"

# Dictionary of misspellings
spell_filename = "rmh_2012_2017_dev_amt5"

# Classifier and threshold
model_filename = "calibrated_lgbm_rmh_2012_2017_dev_amt5"

In [ ]:
# Load the ED vocabulary
vocab = load_vocab(vocab_filename)

# Load ED word frequency list
# word_list = load_word_list(vocab_filename)

# Load the dictionary of corrected misspellings
# misspelled_dict = load_misspelled_dict(vocab_filename)

# Load a pre-trained model and threshold
# model, thresh = load_model(model_filename)

In [ ]:
def check_note_length(df):
    print("Across the dataset, the average note length is %d (+/- %d)" % (df.length.mean(), df.length.std()))
    print("For notes negative for SH, the average note length is %d (+/- %d)" % (df[df.SH==0].length.mean(), df[df.SH==0].length.std()))
    print("For notes positive for SH, the average note length is %d (+/- %d)" % (df[df.SH==1].length.mean(), df[df.SH==1].length.std()))

    sns.kdeplot(x='length', data=df, hue='SH', fill=True, common_norm=False);

# def check_unknown_words(notes, vocab):
#     # Find and count OOV words
#     unknown_tokens = find_unknown_tokens(notes, vocab)
#     # Top 1% OOV words by frequency
#     q = np.quantile(list(unknown_tokens.values()), q=0.99)
#     print("Top 1% OOV words by frequency:", q)
#     print("There are %d such words." % sum([v >= q for v in unknown_tokens.values()]))
#     print([k for k,v in unknown_tokens.items() if v >= q])


### Load data

In [ ]:
# Dataset used for analysis
unseen_data_filename = "lvrh_2012_2022" # "rmh_2012_2017_dev_amt5" | "rmh_2012_2017_test" | "rmh_2018_2022" | "lvrh_2012_2022"

# Load data for analysis
df = pd.read_csv("../datasets/" + unseen_data_filename + "_normalised.csv", 
                 converters={'triage_note': str, 
                             'preprocessed_triage_note': str, 
                             'tokenized_triage_note': str, 
                             'corrected_triage_note': str, 
                             'normalised_triage_note': str,
                             'entities': str})

df = df[df.year < 2018].copy()
print(df.shape)
df.head(1)

In [ ]:
dev_tokens = count_tokens(df.entities, valid_tokens_only=True, return_dict=True)

In [ ]:
test_tokens = count_tokens(df.entities, valid_tokens_only=True, return_dict=True)

In [ ]:
ext_tokens = count_tokens(df.entities, valid_tokens_only=True, return_dict=True)

In [ ]:
sum([t in dev_tokens for t in test_tokens])

In [ ]:
sum([t in dev_tokens for t in ext_tokens])

In [ ]:
32107/75235*100

In [ ]:
count_tokens(df.tokenized_triage_note, valid_tokens_only=True, tokens_in_vocab=True, vocab=vocab)

In [ ]:
count_tokens(df.corrected_triage_note, valid_tokens_only=True, tokens_in_vocab=True, vocab=vocab)

In [ ]:
check_note_length(df)

### Text normalisation

In [ ]:
n = 100
m = 50000

In [ ]:
token_counts = count_tokens(df.triage_note, valid_tokens_only=True, return_dict=True, verbose=False)

In [ ]:
# Proba ~ freq
token_counts = count_tokens(df.triage_note, valid_tokens_only=True, return_dict=True, verbose=False)

samples = np.random.choice(a=np.array(list(token_counts.keys())), 
                          size=(n, m), 
                          p=np.array(list(token_counts.values())) / sum(token_counts.values())
                          )

# # Uniform distribution
# tokens = []
# df.triage_note.apply(lambda x: [tokens.append(token) for token in x.split() if is_valid_token(token)])

# samples = np.random.choice(tokens, size=(n, m))

In [ ]:
raw_unique = [np.unique(s).shape for s in samples]
print("# unique in raw notes: %d (+/- %d)" % (np.mean(raw_unique), np.std(raw_unique)))

In [ ]:
# # Proba ~ freq
# token_counts = count_tokens(df.tokenized_triage_note, valid_tokens_only=True, return_dict=True, verbose=False)

# samples = np.random.choice(a=np.array(list(token_counts.keys())), 
#                           size=(n, m), 
#                           p=np.array(list(token_counts.values())) / sum(token_counts.values()))

# Uniform distribution
tokens = []
df.tokenized_triage_note.apply(lambda x: [tokens.append(token) for token in x.split() if is_valid_token(token)])

samples = np.random.choice(tokens, size=(n, m))

In [ ]:
tok_unique = [np.unique(s).shape for s in samples]
print("# unique in tokenised notes: %d (+/- %d)" % (np.mean(tok_unique), np.std(tok_unique)))

In [ ]:
tok_known = [len(vocab.intersection(s)) for s in samples]
print("# known words in tokenised notes: %d (+/- %d)" % (np.mean(tok_known), np.std(tok_known)))

In [ ]:
tok_missp = [len(set(misspelled_dict.keys()).intersection(s)) for s in samples]
print("# misspellings in tokenised notes: %d (+/- %d)" % (np.mean(tok_missp), np.std(tok_missp)))

In [ ]:
# # Proba ~ freq
# token_counts = count_tokens(df.corrected_triage_note, valid_tokens_only=True, return_dict=True, verbose=False)

# samples = np.random.choice(a=np.array(list(token_counts.keys())), 
#                           size=(n, m), 
#                           p=np.array(list(token_counts.values())) / sum(token_counts.values()))

# Uniform distribution
tokens = []
df.corrected_triage_note.apply(lambda x: [tokens.append(token) for token in x.split() if is_valid_token(token)])

samples = np.random.choice(tokens, size=(n, m))

In [ ]:
corr_unique = [np.unique(s).shape for s in samples]
print("# unique in corrected notes: %d (+/- %d)" % (np.mean(corr_unique), np.std(corr_unique)))

In [ ]:
sum([t not in vocab for t in np.unique(samples[0])])

In [ ]:
sum([t in vocab for t in np.unique(samples[0])])

In [ ]:
sum([t not in samples[0] for t in vocab])

In [ ]:
len(vocab.difference(samples[0]))

In [ ]:
len(set(samples[0]).difference(vocab))

In [ ]:
sum(np.in1d(samples[0], vocab))

In [ ]:
def calculate_language_stats(df):
    stats = {
        'raw_unique': [], 
        'tok_unique': [],
        'tok_reduction': [],
        'tok_known': [],
        'tok_missp': [],
        'tok_oov': [],
        'cor_unique': [],
        'cor_reduction': [],
        'norm_unique': [],
        'norm_reduction': [],
    }
    for i in range(100):
        tmp = df[(df.length >= min_len) & (df.length <= max_len)].sample(10000).copy()
        # Raw notes
        token_counts = count_tokens(tmp.triage_note, valid_tokens_only=True, return_dict=True, verbose=False)
        # unique tokens
        raw_unique = len(token_counts.keys())
        stats['raw_unique'].append(raw_unique)
    
        # Tokenised notes
        token_counts = count_tokens(tmp.tokenized_triage_note, valid_tokens_only=True, return_dict=True, verbose=False)
        # unique tokens
        tok_unique = len(token_counts.keys())
        stats['tok_unique'].append(tok_unique)
        # reduction
        stats['tok_reduction'].append(100 - tok_unique / raw_unique * 100)
        # known to vocab
        unknown_tokens = [k for k in token_counts.keys() if k not in vocab]
        stats['tok_known'].append(tok_unique - len(unknown_tokens))
        # misspelled and OOV
        oov, missp = np.bincount([k in misspelled_dict for k in unknown_tokens])
        stats['tok_missp'].append(missp)
        stats['tok_oov'].append(oov)
        

        # Corrected notes
        token_counts = count_tokens(tmp.corrected_triage_note, valid_tokens_only=True, return_dict=True, verbose=False)
        # unique tokens
        cor_unique = len(token_counts.keys())
        stats['cor_unique'].append(cor_unique)
        # reduction
        stats['cor_reduction'].append(100 - cor_unique / tok_unique * 100)

        # Normalised notes
        token_counts = count_tokens(tmp.entities, valid_tokens_only=True, return_dict=True, verbose=False)
        # unique tokens
        norm_unique = len(token_counts.keys())
        stats['norm_unique'].append(norm_unique)
        # reduction
        stats['norm_reduction'].append(100 - norm_unique / cor_unique * 100)

    print("Raw notes")
    print("# unique in raw notes: %d (+/- %d)" % (np.mean(stats['raw_unique']), np.std(stats['raw_unique'])))
    print()
    print("Pre-processing & tokenisation")
    print("# unique in tokenised notes: %d (+/- %d)" % (np.mean(stats['tok_unique']), np.std(stats['tok_unique'])))
    print("pre-processing & tokenisation reduce dimensionality by %d%% (+/- %d)" % (np.mean(stats['tok_reduction']), np.std(stats['tok_reduction'])))
    print("# known words in tokenised notes: %d (+/- %d)" % (np.mean(stats['tok_known']), np.std(stats['tok_known'])))
    print("# misspellings in tokenised notes: %d (+/- %d)" % (np.mean(stats['tok_missp']), np.std(stats['tok_missp'])))
    print("# OOV words in tokenised notes: %d (+/- %d)" % (np.mean(stats['tok_oov']), np.std(stats['tok_oov'])))
    print()
    print("Spelling correction")
    print("# unique in corrected notes: %d (+/- %d)" % (np.mean(stats['cor_unique']), np.std(stats['cor_unique'])))
    print("spelling correction reduces dimensionality by %d%% (+/- %d)" % (np.mean(stats['cor_reduction']), np.std(stats['cor_reduction'])))
    print()
    print("Final")
    print("# unique in corrected notes: %d (+/- %d)" % (np.mean(stats['norm_unique']), np.std(stats['norm_unique'])))
    print("Normalisation reduces dimensionality by %d%% (+/- %d)" % (np.mean(stats['norm_reduction']), np.std(stats['norm_reduction'])))

In [ ]:
calculate_language_stats(df)

**Token counts**

In [ ]:
count_tokens(df.triage_note, valid_tokens_only=True)
count_tokens(df.preprocessed_triage_note, valid_tokens_only=True)
count_tokens(df.tokenized_triage_note, valid_tokens_only=True)
count_tokens(df.corrected_triage_note, valid_tokens_only=True)
count_tokens(df.normalised_triage_note, valid_tokens_only=True)
count_tokens(df.entities, valid_tokens_only=True)

**Raw notes**

In [ ]:
# Number of unique words and lexical diversity
get_language_stats(df.triage_note)

**Before spelling correction**

In [ ]:
get_language_stats(df.tokenized_triage_note)

**After spelling correction**

In [ ]:
get_language_stats(df.corrected_triage_note)

### Uncorrected words

In [ ]:
# check_unknown_words(df.normalised_triage_note, vocab)

In [ ]:
%%time
unknown_tokens = find_correct_spelling(unknown_tokens, misspelled_dict=misspelled_dict)

In [ ]:
%%time
unknown_tokens = find_correct_spelling(unknown_tokens, 
                                       misspelled_dict=misspelled_dict, 
                                       word_list=word_list)

### Corrected misspellings and  OOV tokens

In [ ]:
misspelled, oov = split_misspelled_oov(unknown_tokens)

In [ ]:
sorted(misspelled.items(),  key=lambda item: item[1], reverse=False)

In [ ]:
sorted(oov.items(),  key=lambda item: item[1], reverse=True)

In [ ]:
misspelled_dict = dict(misspelled_dict, **misspelled)
len(misspelled_dict)

In [ ]:
with open("../datasets/spelling_correction/" + unseen_data_filename + "_misspelled_dict.pickle", 'wb') as f:
    pickle.dump(misspelled_dict, f)

In [ ]:
[k for k,v in misspelled_dict.items() if v==None]

# Threshold

In [ ]:
# Dataset used for analysis
unseen_data_filename = "lvrh_2012_2022" # "rmh_2012_2017_dev_amt5" | "rmh_2012_2017_test" | "rmh_2018_2022" | "lvrh_2012_2022"

# Load data for analysis
df = pd.read_csv("../datasets/" + unseen_data_filename + "_predicted.csv", 
                 converters={'triage_note': str, 
                             'preprocessed_triage_note': str, 
                             'tokenized_triage_note': str, 
                             'corrected_triage_note': str, 
                             'normalised_triage_note': str,
                             'entities': str})

# Remove 2022
df = df[df.year < 2022].copy()
print(df.shape)
df.head(1)

In [ ]:
evaluate_classification(df.SH, df.prediction)

In [ ]:
thresh = []
# n = [100, 250, 500, 750, 1000, 1500, 2000, 3000, 5000, 7500, 10000, 15000, 20000, 25000, 30000, 40000, 50000, 100000]
n = range(100, 300000, 250)
for i in n:
    # tmp = df[['SH', 'probability']].sample(i).copy()
    tmp = df[['SH', 'probability']].iloc[:i].copy()
    thresh.append(select_threshold(tmp.SH, tmp.probability, verbose=False))

In [ ]:
# Randomly sampled
sns.lineplot(x=n, y=thresh)

In [ ]:
# Consequetive
sns.lineplot(x=n, y=thresh)

In [ ]:
evaluate_classification(df.SH, threshold_proba(df.probability, thresh))

# Features

In [ ]:
with open("../models/selected_fts_" + dev_data_filename + "_amt5.txt", 'r') as f:
    selected_features = f.read().split()
len(selected_features)

In [ ]:
with open("../models/selected_fts_" + dev_data_filename + "2.txt", 'r') as f:
    selected_features2 = f.read().split()
len(selected_features2)

In [ ]:
fts_v1 = [ft for ft in selected_features if not ft in selected_features2] #17

In [ ]:
fts_v2 = [ft for ft in selected_features2 if not ft in selected_features] #27

In [ ]:
stopwords = get_stopwords()
len(stopwords)

In [ ]:
[ft for ft in fts_v2 if ft in stopwords]

In [ ]:
# ED vocabulary
vocab_filename = "rmh_2012_2017_dev_amt5"

# Selected features
vectorizer_filename = "rmh_2012_2017_dev_amt5"

In [ ]:
# Load the ED vocabulary
vocab = load_vocab(vocab_filename)

# Load the list of selected features
vectorizer = load_vectorizer(vectorizer_filename)

In [ ]:
selected_fts = frozenset(vectorizer.df_features.feature.tolist())
len(selected_fts)

In [ ]:
# Dataset used for analysis
unseen_data_filename = "lvrh_2012_2022" # "rmh_2012_2017_dev_amt5" | "rmh_2012_2017_test" | "rmh_2018_2022" | "lvrh_2012_2022"

# Load data for analysis
df = pd.read_csv("../datasets/" + unseen_data_filename + "_normalised.csv", 
                 converters={'triage_note': str, 
                             'preprocessed_triage_note': str, 
                             'tokenized_triage_note': str, 
                             'corrected_triage_note': str, 
                             'normalised_triage_note': str,
                             'entities': str})
print(df.shape)
df.head(1)

In [ ]:
tokens = count_tokens(df.entities, return_dict=True)
sum([ft in tokens for ft in selected_fts])

In [ ]:
vectorizer.fit(df.entities, df.SH)
vectorizer.df_features

In [ ]:
vectorizer.df_features[~vectorizer.df_features.feature.isin(selected_fts)]

In [ ]:
sns.histplot(x='probability', data=df, stat='probability', bins=25);
plt.ylim([0,0.005])

# Predictions

In [ ]:
def load_dataset(filename):
    # Load data for analysis
    df = pd.read_csv("../datasets/" + filename + "_predicted.csv")

    # Convert to datetime and extract year and quarter
    df.arrival_date = pd.to_datetime(df.arrival_date)
    df['year'] = df.arrival_date.dt.year
    df['quarter'] = df.arrival_date.dt.to_period('Q')

    # Remove 2022
    df = df[df.year < 2022].copy()

    return df

In [ ]:
# Dataset used for analysis
unseen_data_filename = "rmh_2018_2022"
df1 = load_dataset(unseen_data_filename)
df1['hospital'] = 'RMH'

unseen_data_filename = "lvrh_2012_2022"
df2 = load_dataset(unseen_data_filename)
df2['hospital'] = 'LRH'

df = pd.concat([df1, df2], axis=0)

print(df.shape)
df.head()

In [ ]:
# # Plot curves
# plot_curves(df.SH, df.probability, filename=unseen_data_filename)
# # Evaluate classification on the whole dataset
# evaluate_classification(df.SH, df.prediction, filename=unseen_data_filename)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 6)
df.quarter = df.quarter.astype('category')
xlim = df.quarter.cat.codes.max() + 1

sns.lineplot(x=range(0, xlim), y=0.85, ls='--', 
             color=sns.color_palette('Paired')[0], label="Development set")
sns.lineplot(x=range(0, xlim), y=0.86, ls='--', 
             color=sns.color_palette('Paired')[2], label="Test set")
sns.lineplot(x=range(0, xlim), y=0.83, ls='--', 
             color=sns.color_palette('Paired')[8], label="Prospective validation set")
sns.lineplot(x=range(0, xlim), y=0.78, ls='--', 
             color=sns.color_palette('Paired')[4], label="External validation set")

plt.xticks(rotation=45, 
           ticks=range(0, xlim, 2), 
           labels=[format_quarter(q) for q in df.quarter.cat.categories.astype(str) if q.endswith('1') or q.endswith('3')]);

sns.lineplot(df[df.hospital=='RMH'].groupby(df[df.hospital=='RMH'].quarter.cat.codes, observed=False).apply(
    lambda x: calculate_auc(x.SH, x.probability, method='pr')), 
             color=sns.color_palette('Paired')[9], lw=2, label="RMH");

sns.lineplot(df[df.hospital=='LRH'].groupby(df[df.hospital=='LRH'].quarter.cat.codes, observed=False).apply(
    lambda x: calculate_auc(x.SH, x.probability, method='pr')), 
             color=sns.color_palette('Paired')[5], lw=2, label="LRH");

plt.ylim([0.45, 1.05]);
plt.ylabel("PR AUC");
plt.savefig("../results/Performance by quarter.jpeg", bbox_inches='tight', dpi=300);

In [ ]:
sns.lineplot(x=range(0, xlim), y=0.53, ls='--', 
             color=sns.color_palette('Set1')[1], label="PPV")
sns.lineplot(x=range(0, xlim), y=0.81, ls='--', 
             color=sns.color_palette('Set1')[2], label="Sensitivity")
sns.lineplot(x=range(0, xlim), y=0.99, ls='--', 
             color=sns.color_palette('Set1')[3], label="Specificity")

sns.lineplot(df[df.hospital=='LRH'].groupby(df[df.hospital=='LRH'].quarter.cat.codes, 
                                            observed=False).apply(lambda x: 
                                                                  precision_score(x.SH, x.prediction)), 
             color=sns.color_palette('Set1')[1], lw=2, label="PPV");

sns.lineplot(df[df.hospital=='LRH'].groupby(df[df.hospital=='LRH'].quarter.cat.codes, 
                                            observed=False).apply(lambda x: 
                                                                  recall_score(x.SH, x.prediction, pos_label=1)), 
             color=sns.color_palette('Set1')[2], lw=2, label="Sensitivity");

sns.lineplot(df[df.hospital=='LRH'].groupby(df[df.hospital=='LRH'].quarter.cat.codes, 
                                            observed=False).apply(lambda x: 
                                                                  recall_score(x.SH, x.prediction, pos_label=0)), 
             color=sns.color_palette('Set1')[3], lw=2, label="Specificity");

In [ ]:
recall_score(df[(df.hospital=='LRH') & (df.quarter=='2021Q4')].SH, 
             df[(df.hospital=='LRH') & (df.quarter=='2021Q4')].prediction, pos_label=1).round(2)

# Reviewed predicitons

In [ ]:
unseen_data_filename = "lvrh_2012_2022"

df = pd.read_csv("../datasets/" + unseen_data_filename + "_predicted.csv")

# Convert to datetime and extract year and quarter
df.arrival_date = pd.to_datetime(df.arrival_date)
df['year'] = df.arrival_date.dt.year
df['quarter'] = df.arrival_date.dt.to_period('Q')

In [ ]:
df_reviewed = pd.read_csv("../datasets/" + unseen_data_filename + "_reviewed.csv")
df_reviewed.rename(columns={'Revised': 'revised', 
                            'Recoded_SH': 'recoded_SH', 
                            'Recoded_SI': 'recoded_SI',
                            'Recoded_AOD_OD': 'recoded_AOD_OD', 
                            'Comment': 'comment'}, 
                            inplace=True)
df = pd.concat([
    df, df_reviewed[['revised', 'recoded_SH', 'recoded_SI', 'recoded_AOD_OD', 'comment']]
    ], axis=1)

# Remove 2022
df = df[df.year < 2018].copy()

print(df.shape)
df.head(1)

### Reviewed notes

In [ ]:
df.for_review.sum()

In [ ]:
df[df.for_review==1].prediction_class.value_counts()

In [ ]:
df[df.for_review==1].year.value_counts()

In [ ]:
(df[df.for_review==1].quarter.value_counts() == 6).all()

In [ ]:
df.revised.sum()

In [ ]:
df[['recoded_SH', 'recoded_SI', 'recoded_AOD_OD']].sum()

### Revised false-positives (Neg pred Pos -> Pos)
**Self-harm**

In [ ]:
df[(df.prediction_class=='FP') & (df.revised==1)]

In [ ]:
df[(df.prediction_class=='FP') & (df.revised==1)].recoded_SH.sum()

In [ ]:
6/72, 11/120

In [ ]:
df[(df.prediction_class=='FP') & (df.revised==1) & (df.recoded_SH==1)].probability.describe()

**Suicidal ideation**

In [ ]:
df.loc[(df.prediction_class=='FP') & (df.revised==1)].recoded_SI.sum()

### Revised false-negatives (Pos pred Neg -> Neg)

In [ ]:
df[(df.prediction_class=='FN') & (df.revised==1)]

In [ ]:
(df[(df.prediction_class=='FN') & (df.revised==1)].recoded_SH.fillna(0)==0).sum()

In [ ]:
5/72, 12/120

In [ ]:
df.loc[(df.prediction_class=='FN') & (df.revised==1)].probability.describe()

**Plot probability distributions**

In [ ]:
# sns.histplot(x='probability', data=df[df.prediction_class=='TN'], 
#              binwidth=0.035, color=sns.color_palette('Paired')[1], label='TN');
# sns.histplot(x='probability', data=df[df.prediction_class=='FN'], 
#              binwidth=0.035, color=sns.color_palette('Paired')[0], label='FN');
# sns.histplot(x='probability', data=df[df.prediction_class=='TP'], 
#              binwidth=0.035, color=sns.color_palette('Paired')[3], label='TP');
# sns.histplot(x='probability', data=df[df.prediction_class=='FP'], 
#              binwidth=0.035, color=sns.color_palette('Paired')[2], label='FP');


# plt.axvline(thresh, 0, 1, color=sns.color_palette('Paired')[5], ls='--', label='Original\nthreshold');
# # plt.axvline(thresh_adj, 0, 1, color=sns.color_palette('Paired')[9], ls='--', label='Adjusted\nthreshold');
# plt.legend();
# plt.xlim([0,1]);
# plt.ylim([0,2200]);
# plt.xlabel("Predicted probability");
# plt.ylabel("Count");
# # plt.savefig("../results/Adjusted threshold.jpeg", bbox_inches='tight', dpi=300);

In [ ]:
df['review_outcome'] = ""
df.loc[(df.prediction_class=='FP') & (df.for_review==1) & (df.recoded_SH!=1), 'review_outcome'] = "Confirmed FP"
df.loc[(df.prediction_class=='FP') & (df.for_review==1) & (df.revised==1) & (df.recoded_SH==1), 'review_outcome'] = "Revised TP"
df.loc[(df.prediction_class=='FN') & (df.for_review==1) & (df.revised==0), 'review_outcome'] = "Confirmed FN"
df.loc[(df.prediction_class=='FN') & (df.for_review==1) & (df.revised==1), 'review_outcome'] = "Revised TN"
df.review_outcome = df.review_outcome.astype('category').cat.set_categories(["Confirmed FN", "Revised TN", 
                                                                             "Confirmed FP", "Revised TP"])
df.review_outcome.value_counts().sort_index()

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)

thresh = 0.36

sns.histplot(x='probability', data=df[df.review_outcome=='Confirmed FN'], 
             binwidth=0.035, binrange=(0,thresh), color=sns.color_palette('Paired')[0], label='Confirmed FN');
sns.histplot(x='probability', data=df[df.review_outcome=='Revised TN'], 
             binwidth=0.035, binrange=(0,thresh), color=sns.color_palette('Paired')[1], label='Revised TN');
sns.histplot(x='probability', data=df[df.review_outcome=='Confirmed FP'], 
             binwidth=0.035, binrange=(thresh,1), color=sns.color_palette('Paired')[2], label='Confirmed FP');
sns.histplot(x='probability', data=df[df.review_outcome=='Revised TP'], 
             binwidth=0.035, binrange=(thresh,1), color=sns.color_palette('Paired')[3], label='Revised TP');

plt.legend();
plt.xlim([0,1]);
plt.xlabel("Predicted probability");
plt.ylabel("Count");
plt.savefig("../results/Review results.jpeg", bbox_inches='tight', dpi=300);

In [ ]:
# Evaluate classification
evaluate_classification(df.SH, df.prediction, filename=None)

In [ ]:
counts = df.prediction_class.value_counts()
counts

In [ ]:
# PPV
fp2tp = counts.FP * 6/72
print("%d notes will be reviewed as true positive" % fp2tp)
(counts.TP + fp2tp) / (counts.TP + counts.FP)

In [ ]:
# Sensitivity
fn2tn = counts.FN * 5/72
print("%d notes will be reviewed as true negative" % fn2tn)
(counts.TP + fp2tp) / (counts.TP + fp2tp + counts.FN - fn2tn)

In [ ]:
# Specificity
(counts.TN + fn2tn) / (counts.TN + counts.FN)

In [ ]:
83 - (83-57)/2